## Notebook principal

Primero se busca trabajar con 3 libros de texto en formato PDF, los cuales son:  
Codigo limpio - Robert Cecil Martin  
Ingenieria de software - Pressman  
Ingenieria de software - Sommerville

In [21]:
%pip install PyPDF2
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [22]:
from PyPDF2 import PdfReader
from tqdm.auto import tqdm
from os import makedirs, listdir, path, getcwd

BASE_DIR = getcwd()
DATA_DIR = path.join(BASE_DIR, "data")
PDF_DIR = path.join(DATA_DIR, "pdf")
TXT_DIR = path.join(DATA_DIR, "txt")

Una vez teniendo los libros en formato PDF, se procede a convertirlos a formato TXT, para poder trabajar con ellos.
La conversion ocurre pagina por pagina.

In [59]:
# Crea el directorio si no existe
makedirs(TXT_DIR, exist_ok=True)

In [68]:
pdf_files = [f for f in listdir(PDF_DIR) if f.lower().endswith('.pdf')]
for pdf in pdf_files:
    pdf_reader = PdfReader((path.join(PDF_DIR, pdf)))
    progress_bar = tqdm(range(len(pdf_reader.pages)))
    progress_bar.set_description(f'Processing {pdf}')
    text = ' '.join([pdf_reader.pages[page].extract_text().replace('\n', ' ') for page in progress_bar])
    with open(path.join(TXT_DIR, f'{pdf[:-4]}.txt'), 'w') as f:
        f.write(text)

Processing codigo_limpio_robert_cecil_martin.pdf: 100%|██████████| 644/644 [00:13<00:00, 47.41it/s] 
Processing ingenieria_de_software_sommerville.PDF: 100%|██████████| 792/792 [00:03<00:00, 200.58it/s]
Processing ingenieria_de_software_pressman.PDF: 100%|██████████| 810/810 [00:08<00:00, 92.95it/s] 


Una vez teniendo todos los archivos .txt se debe obtener el dataset de STS lleno de oraciones relacionadas a la ingenieria de software y un puntaje entre 0 y 1 que valore la similitud entre las oraciones.

Se buscara hacer uso de algun modelo para extraer las oraciones del archivo .txt

In [27]:
%pip install huggingface_hub transformers datasets  

Note: you may need to restart the kernel to use updated packages.


Buscando los modelos en espanol para extraer automaticamente las oraciones de ingenieria de software encontre este modelo: [hiiamsid/sentence_similarity_spanish_es](https://huggingface.co/hiiamsid/sentence_similarity_spanish_es) el cual potencialmente sera usado para realizarle fine-tuning y asi obtener un modelo final.

Se busca obtener cada oracion del archivo .txt principal de cada libro e incluirla en un archivo .json.

Primero se limpia el texto de cada archivo .txt, se eliminan los saltos de linea, espacios en blanco, caracteres especiales, palabras que no aportan informacion, palabras repetidas.

In [28]:
import re

def remove_useless_text(text):
    # Regular expressions for detecting useless text patterns
    patterns = [
        r'>',  # Remove bullet point indicators
        r'\d+\w*',  # Remove numbers or digits combined with letters
        r'\b\w\b',  # Remove single-letter words
        r'\b\w{1,2}\b',  # Remove words with one or two characters
        r'www\.[^\s]+',  # Remove URLs
    ]

    # Apply each pattern sequentially to remove useless text
    for pattern in patterns:
        text = re.sub(pattern, '', text)

    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# IDEA RAPIDA.

USAR UN TOKENIZER EN ESPANOL PARA TOKENIZAR POR SUBWORD Y LIMPIAR EL TEXTO CON ESO COMO ALTERNATIVA A remove_useless_text()

In [2]:
%pip install torch

  Using cached torch-2.0.1-cp310-cp310-manylinux1_x86_64.whl (619.9 MB)
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached networkx-3.1-py3-none-any.whl (2.1 MB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cached nvidia_cuda_cupti_cu11-11.7.101-py3-none-manylinux1_x86_64.whl (11.8 MB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
  Using cached nvidia_cufft_cu11-10.9.0.58-py3-none-manylinux1_x86_64.whl (168.4 MB)
  Using cached nvidia_curand_cu11-10.2.10.91-py3-none-manylinux1_x86_64.whl (54.6 MB)
  Using cached nvidia_cusolver_cu11-11.4.0.1-2-py3-none-manylinux1_x86_64.whl (102.6 MB)
  Using cached nvidia_cusparse_cu11-11.7.4.91-py3-none-manylinux1_x86_64.whl (173.2 MB)
  Using cached nvidia_nccl_cu11-2.14

In [51]:
for filename in listdir(TXT_DIR):
    with open(filename, 'r', encoding='utf-8') as file:
        text = file.read()
        cleaned_text = remove_useless_text(text)
        print(len(cleaned_text), len(text))
        # Write the cleaned text to a new file
        cleaned_file_path = path.join(TXT_DIR, f'cleaned_{filename}')
        with open(cleaned_file_path, 'w', encoding='utf-8') as cleaned_file:
            cleaned_file.write(cleaned_text)


0 0
1908781 1908781
670110 670110


In [37]:

for filename in listdir(TXT_DIR):
    if not any(char.isdigit() for char in filename):
        # por la convencion de nombres que se tomo, estos archivos son los que contienen todo el texto de su respectivo pdf
        file_path = path.join(TXT_DIR, filename)
        with open(path.join(TXT_DIR, filename), 'rw', encoding='utf-8') as file:
            text = file.read()
            cleaned_text = remove_useless_text(text)
            file.write
            # Do something with the cleaned text, such as write it to a new file
            with open(file_path, 'w', encoding='utf-8') as cleaned_file:
                cleaned_file.write(cleaned_text)

UnsupportedOperation: not readable

Ya con un archivo que contiene texto "limpio" se obtienen las oraciones de cada libro y se incluyen en un archivo .json.

In [8]:
%pip install nltk  

Note: you may need to restart the kernel to use updated packages.


In [9]:
# import nltk
# import json

# # Descargar el tokenizador de oraciones de NLTK
# nltk.download('punkt')

# with open('/home/abraham/personal/sts-swe/data/txt/cleaned_codigo_limpio_robert_cecil_martin.txt', 'r') as file:
#     text = file.read()

# sentences = nltk.sent_tokenize(text)
# meaningful_sentences = []
# for sentence in sentences:
#     # Si la oración tiene más de 5 palabras, la consideramos significativa
#     if len(sentence.split()) > 5:
#         # Decodificar la oración para reemplazar los escape sequences con los caracteres correspondientes
#         sentence = sentence.encode().decode('unicode_escape')
#         meaningful_sentences.append(sentence)

# # Crear un objeto JSON y agregar las oraciones significativas como elementos del objeto
# data = {}
# data['sentences'] = meaningful_sentences

# with open('meaningful_sentences.json', 'w') as file:
#     json.dump(data, file, indent=4)

In [10]:
%pip install spacy
!python -m spacy download es_core_news_sm

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 54.5 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [20]:
import spacy
import json


# Load Spanish tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("es_core_news_sm")  
# you can change "es_core_news_sm" to "es_core_news_md" if you have downloaded the medium Spanish model

# Increase the max_length limit to 2000000
nlp.max_length = 2000000

# Read the text file
with open('/home/abraham/personal/sts-swe/data/txt/cleaned_codigo_limpio_robert_cecil_martin.txt', 'r') as file:
    text = file.read()

# Process whole documents
doc = nlp(text)

# Tokenize sentence
sentences = [sent.text for sent in doc.sents]


[] http:// <<


In [15]:
meaningful_sentences = []
for sentence in sentences:
    # Si la oración tiene más de 5 palabras, la consideramos significativa
    if len(sentence.split()) > 5:
        # Decodificar la oración para reemplazar los escape sequences con los caracteres correspondientes
        sentence = sentence.encode().decode('unicode_escape')
        meaningful_sentences.append(sentence)

# Crear un objeto JSON y agregar las oraciones significativas como elementos del objeto
data = {}
data['sentences'] = meaningful_sentences

with open('meaningful_sentences.json', 'w') as file:
    json.dump(data, file, indent=4)

Se opta por usar los embeddings del modelo anteriormente mencionado para asi encontrar el contexto de cada oracion.